In [1]:
!pip install -q pgmpy

# Noisy-OR model

## Deterministic OR

It is easy to see that an OR between many variables can factorize as a product of tables involving only three variables

$$
\begin{align}\label{eq:or}
 \text{OR}(y|x_1,x_2,x_3) & = \sum_{s\in\{0,1\}} \text{OR}(y|x_3,s)\text{OR}(s|x_1,x_2).
\end{align}
$$

## Noisy OR

The noisy-OR model \[1\] represents a joint probability distribution composed of an effect variable $y$ that has $K$ parents $x_k, k=1,...,K$. For simplicity, we will assume that all of them are binary. It factorizes as

$$
\begin{align}
 P(y=0|x_1,...,x_K) & = P(y=0|L)\prod_{k=1}^{K} P(y=0|x_k)\\
 & = (1-\lambda_0)\prod_{k=1}^{K} (1-\lambda_k)^{x_k},
\end{align}
$$

where $\lambda_k$ is used to parameterize the probability that the parent $k$, if present, could alone determine the effect to have a positive outcome. The parameter $\lambda_0 = P(y=1|L)$, also known as leak probability, is the probability that $y$ occurs by means other than the specified parents.

Note that since $x_k$ appears in the exponent, when the parent is not active ($x_k=0$), the corresponding term is one, so the probability is not affected by $\lambda_k$. On the contrary, when $x_k=1$, the lower the value of $\lambda_k$, the less likely will be that the effect is present, and vice-versa.

## Using exponentially large OR tables
A simple way to represent this model consists of $K$ factors (one for each parent $x_k$) that have a direct relation with a hidden variable $z_k$. These potentials encode the local probability that the effect is active or not, given the value of the cause only:
<table>
    <tr><th>$x_k$</th><th>$z_k$</th><th>$P(z_k|x_k)$</th></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> $1$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> $(1-\lambda_k)$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> $\lambda_k$ </td></tr>
</table>

An additional leak factor $L$ is also introduced to account for the term $(1-\lambda_0)$ and $L$ is usually set to $1$ (i.e. $P(L=1)=1$).

All these factors are combined using a deterministic OR factor. The size of this gate grows exponentially in $K$. For example, for $K=3$:
<table>
    <tr><th> $z_0$ </th> <th> $z_1$ </th> <th> $z_2$ </th> <th> $z_3$ </th> <th> $P(y=0|z_0,z_1,z_2,z_3)$ </th></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> 0 </td> <td> 0 </td> <td> $1$ </td></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> 1 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 0 </td> <td> 1 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> 0 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> 1 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 0 </td> <td> 1 </td> <td> 1 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> 0 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> 1 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 0 </td> <td> 1 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> 0 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> 0 </td> <td> 1 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> 1 </td> <td> 0 </td> <td> $0$ </td></tr>
    <tr><td> 1 </td> <td> 1 </td> <td> 1 </td> <td> 1 </td> <td> $0$ </td></tr>
</table>
<br><center>Table 1: A deterministic OR factor: full table for four inputs</center>

Note that we also need to specify the prior probabilities for $x_k, \forall k$. For simplicity, we will assume they are uniform.

Create a model like that and experiment with different values of $\boldsymbol{\lambda}$. For example, assume $\lambda_0=10^{-5},\lambda_1=0.9,\lambda_2=0.5,\lambda_3=0.99$. The corresponding graphical model is <br> <img src="https://raw.githubusercontent.com/guillermoim/resources/main/noisyor.png" width=350 height=230>

Construct this model using python.

In [2]:
import numpy as np
from pgmpy.factors.discrete import DiscreteFactor, TabularCPD

#Define variables and domains
variables = ["L", "x_1", "x_2", "x_3", "z_0", "z_1", "z_2", "z_3", "y"]
domains = {"L": ["0", "1"],
           "x_1": ["0", "1"],
           "x_2": ["0", "1"],
           "x_3": ["0", "1"],
           "z_0": ["0", "1"],
           "z_1": ["0", "1"],
           "z_2": ["0", "1"],
           "z_3": ["0", "1"],
           "y": ["0", "1"]}

p = dict()    # use it to store out potentials

p["x_1"] = DiscreteFactor(variables=["x_1"],
                        cardinality=[len(domains["x_1"])],
                        values=[0.5, 0.5],
                        state_names=domains)

p["x_2"] = DiscreteFactor(variables=["x_2"],
                        cardinality=[len(domains["x_2"])],
                        values=[0.5, 0.5],
                        state_names=domains)

p["x_3"] = DiscreteFactor(variables=["x_3"],
                        cardinality=[len(domains["x_3"])],
                        values=[0.5, 0.5],
                        state_names=domains)

lambda_0 = 0.00001
lambda_1 = 0.9
lambda_2 = 0.5
lambda_3 = 0.99

p["L"] = DiscreteFactor(variables=["L"],
                        cardinality=[len(domains["L"])],
                        values=[0, 1],
                        state_names=domains)

p["z_0|L"] = DiscreteFactor(variables=["L", "z_0"],
                        cardinality=[len(domains["L"]), len(domains["z_0"])],
                        values=[1, 0, 1-lambda_0, lambda_0],
                        state_names=domains)  

p["z_1|x_1"] = DiscreteFactor(variables=["x_1", "z_1"],
                        cardinality=[len(domains["x_1"]), len(domains["z_1"])],
                        values=[1, 0, 1-lambda_1, lambda_1],
                        state_names=domains)

p["z_2|x_2"] = DiscreteFactor(variables=["x_2", "z_2"],
                        cardinality=[len(domains["x_2"]), len(domains["z_2"])],
                        values=[1, 0, 1-lambda_2, lambda_2],
                        state_names=domains)

p["z_3|x_3"] = DiscreteFactor(variables=["x_3", "z_3"],
                        cardinality=[len(domains["x_3"]), len(domains["z_3"])],
                        values=[1, 0, 1-lambda_3, lambda_3],
                        state_names=domains)

p["y|z_0,z_1,z_2,z_3"] = DiscreteFactor(variables=["y", "z_0", "z_1", "z_2", "z_3"],
                        cardinality=[len(domains["y"]), len(domains["z_0"]), len(domains["z_1"]), len(domains["z_2"]), len(domains["z_3"])],
                        values=[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                        state_names=domains)

p["y,z_0,z_1,z_2,z_3,L,x_1,x_2,x_3"] = p["y|z_0,z_1,z_2,z_3"] * p["z_0|L"] * p["z_1|x_1"] * p["z_2|x_2"] * p["z_3|x_3"] * p["L"] * p["x_1"] * p["x_2"] * p["x_3"]
print("Sum of values: ", np.sum(p["y,z_0,z_1,z_2,z_3,L,x_1,x_2,x_3"].values))

p["y,x_1,x_2,x_3"] = p["y,z_0,z_1,z_2,z_3,L,x_1,x_2,x_3"].marginalize(["L", "z_0", "z_1", "z_2","z_3"], inplace=False)
print(p["y,x_1,x_2,x_3"])

Sum of values:  1.0
+--------+------+--------+--------+----------------------+
| x_2    | y    | x_3    | x_1    |   phi(x_2,y,x_3,x_1) |
+========+======+========+========+======================+
| x_2(0) | y(0) | x_3(0) | x_1(0) |               0.1250 |
+--------+------+--------+--------+----------------------+
| x_2(0) | y(0) | x_3(0) | x_1(1) |               0.0125 |
+--------+------+--------+--------+----------------------+
| x_2(0) | y(0) | x_3(1) | x_1(0) |               0.0012 |
+--------+------+--------+--------+----------------------+
| x_2(0) | y(0) | x_3(1) | x_1(1) |               0.0001 |
+--------+------+--------+--------+----------------------+
| x_2(0) | y(1) | x_3(0) | x_1(0) |               0.0000 |
+--------+------+--------+--------+----------------------+
| x_2(0) | y(1) | x_3(0) | x_1(1) |               0.1125 |
+--------+------+--------+--------+----------------------+
| x_2(0) | y(1) | x_3(1) | x_1(0) |               0.1238 |
+--------+------+--------+--------+-

Answer the following questions:


1. What is $P_1=P(y=1|x_1=0,x_2=0,x_3=1)$?

In [3]:
p["y|x_1,x_2,x_3"] = p["y,x_1,x_2,x_3"]/p["x_1"]/p["x_2"]/p["x_3"]
P_1 = p["y|x_1,x_2,x_3"].reduce([("y", "1"), ("x_1", "0"), ("x_2", "0"), ("x_3", "1")], inplace=False)
print(P_1)

+---------+
|   phi() |
+=========+
|  0.9900 |
+---------+


2. Argue why $P_2=P(y=1|x_1=0,x_2=1,x_3=0)$ is smaller than $P_1$.

In [4]:
P_2 = p["y|x_1,x_2,x_3"].reduce([("y", "1"), ("x_1", "0"), ("x_2", "1"), ("x_3", "0")], inplace=False)
print(P_2)

+---------+
|   phi() |
+=========+
|  0.5000 |
+---------+


As the variable that is "up" (set to 1) is $x_2$ instead of $x_2$ and $\lambda_2 < \lambda_3$, P2 is smaller than P1.

3. Relate $P_3=P(y=1|x_1=0,x_2=1,x_3=1)$ with $P_2$ and $P_1$.

In [5]:
P_3 = p["y|x_1,x_2,x_3"].reduce([("y", "1"), ("x_1", "0"), ("x_2", "1"), ("x_3", "1")], inplace=False)
print(P_3)

+---------+
|   phi() |
+=========+
|  0.9950 |
+---------+


Following last argument, as now we have both variables set to 1, the probability of y being 1 is greater. Due to the fact that if both things happen, the output will be more likely than if only one of those happens.

4. Relate $P_4=P(y=1|x_1=0,x_2=0,x_3=0)$ with all the previous probabilities.

In [6]:
P_4 = p["y|x_1,x_2,x_3"].reduce([("y", "1"), ("x_1", "0"), ("x_2", "0"), ("x_3", "0")], inplace=False)
print(P_4)

+---------+
|   phi() |
+=========+
|  0.0000 |
+---------+


As none of the variables are set to 1, the probability that the output event happens is close to 0 or 0. Because the probability will be really low.

5. What are the posterior probabilities of each individual parent, if we observe that $y=1$? How do they change if we observe that $y=0$?

In [7]:
p["y"] = p["y,x_1,x_2,x_3"].marginalize(["x_1", "x_2", "x_3"], inplace=False)
p["x_1,x_2,x_3|y"] = p["y,x_1,x_2,x_3"]/p["y"]

print("y = 1:")
p["x_1|y=1"] = p["x_1,x_2,x_3|y"].marginalize(["x_2", "x_3"], inplace=False).reduce([("y", "1")], inplace=False)
print(p["x_1|y=1"])
p["x_2|y=1"] = p["x_1,x_2,x_3|y"].marginalize(["x_1", "x_3"], inplace=False).reduce([("y", "1")], inplace=False)
print(p["x_2|y=1"])
p["x_3|y=1"] = p["x_1,x_2,x_3|y"].marginalize(["x_1", "x_2"], inplace=False).reduce([("y", "1")], inplace=False)
print(p["x_3|y=1"])

p["x_1|y=0"] = p["x_1,x_2,x_3|y"].marginalize(["x_2", "x_3"], inplace=False).reduce([("y", "0")], inplace=False)
print(p["x_1|y=0"])
p["x_2|y=0"] = p["x_1,x_2,x_3|y"].marginalize(["x_1", "x_3"], inplace=False).reduce([("y", "0")], inplace=False)
print(p["x_2|y=0"])
p["x_3|y=0"] = p["x_1,x_2,x_3|y"].marginalize(["x_1", "x_2"], inplace=False).reduce([("y", "0")], inplace=False)
print(p["x_3|y=0"])

y = 1:
+--------+------------+
| x_1    |   phi(x_1) |
+========+============+
| x_1(0) |     0.3924 |
+--------+------------+
| x_1(1) |     0.6076 |
+--------+------------+
+--------+------------+
| x_2    |   phi(x_2) |
+========+============+
| x_2(0) |     0.4561 |
+--------+------------+
| x_2(1) |     0.5439 |
+--------+------------+
+--------+------------+
| x_3    |   phi(x_3) |
+========+============+
| x_3(0) |     0.3710 |
+--------+------------+
| x_3(1) |     0.6290 |
+--------+------------+
+--------+------------+
| x_1    |   phi(x_1) |
+========+============+
| x_1(0) |     0.9091 |
+--------+------------+
| x_1(1) |     0.0909 |
+--------+------------+
+--------+------------+
| x_2    |   phi(x_2) |
+========+============+
| x_2(0) |     0.6667 |
+--------+------------+
| x_2(1) |     0.3333 |
+--------+------------+
+--------+------------+
| x_3    |   phi(x_3) |
+========+============+
| x_3(0) |     0.9901 |
+--------+------------+
| x_3(1) |     0.0099 |
+--------

## Efficient representation

The previous networks contains a factor with exponetial size, which renders it unfeasible for large $K$. Based on the factorization of the deterministic OR shown at the top, can you think of a more efficient way to represent the noisy-OR model?

Answer the following questions:

1. Using the efficient representation, compute the probabilities of the previous subsection and check they are equivalent

In [8]:
#Define variables and domains
variables = ["L", "x_1", "x_2", "x_3", "z_0", "z_1", "z_2", "z_3", "y", "o_1", "o_2"]
domains = {"L": ["0", "1"],
           "x_1": ["0", "1"],
           "x_2": ["0", "1"],
           "x_3": ["0", "1"],
           "z_0": ["0", "1"],
           "z_1": ["0", "1"],
           "z_2": ["0", "1"],
           "z_3": ["0", "1"],
           "y": ["0", "1"],
           "o_1": ["0", "1"],
           "o_2": ["0", "1"]}

p_2 = dict()  # use it to store out potentials

p_2["o_1|z_0,z_1"] = DiscreteFactor(variables=["o_1", "z_0", "z_1"],
                        cardinality=[len(domains["o_1"]), len(domains["z_0"]), len(domains["z_1"])],
                        values=[1, 0, 0, 0, 0, 1, 1, 1],
                        state_names=domains)

p_2["o_2|z_2,z_3"] = DiscreteFactor(variables=["o_2", "z_2", "z_3"],
                        cardinality=[len(domains["o_2"]), len(domains["z_2"]), len(domains["z_3"])],
                        values=[1, 0, 0, 0, 0, 1, 1, 1],
                        state_names=domains)

p_2["y|o_1,o_2"] = DiscreteFactor(variables=["y", "o_1", "o_2"],
                        cardinality=[len(domains["y"]), len(domains["o_1"]), len(domains["o_2"])],
                        values=[1, 0, 0, 0, 0, 1, 1, 1],
                        state_names=domains)

p_2["y,z_0,z_1,z_2,z_3,L,x_1,x_2,x_3,o_1,o_2"] = p_2["y|o_1,o_2"] * p_2["o_1|z_0,z_1"] * p_2["o_2|z_2,z_3"] * p["z_0|L"] * p["z_1|x_1"] * p["z_2|x_2"] * p["z_3|x_3"] * p["L"] * p["x_1"] * p["x_2"] * p["x_3"]
p_2["y,x_1,x_2,x_3"] = p_2["y,z_0,z_1,z_2,z_3,L,x_1,x_2,x_3,o_1,o_2"].marginalize(["L", "z_0", "z_1", "z_2","z_3", "o_1", "o_2"], inplace=False)

p_2["y"] = p_2["y,x_1,x_2,x_3"].marginalize(["x_1", "x_2", "x_3"], inplace=False)
p_2["x_1,x_2,x_3|y"] = p_2["y,x_1,x_2,x_3"]/p_2["y"]

print("y = 1:")
p_2["x_1|y=1"] = p_2["x_1,x_2,x_3|y"].marginalize(["x_2", "x_3"], inplace=False).reduce([("y", "1")], inplace=False)
print(p_2["x_1|y=1"])
p_2["x_2|y=1"] = p_2["x_1,x_2,x_3|y"].marginalize(["x_1", "x_3"], inplace=False).reduce([("y", "1")], inplace=False)
print(p_2["x_2|y=1"])
p_2["x_3|y=1"] = p_2["x_1,x_2,x_3|y"].marginalize(["x_1", "x_2"], inplace=False).reduce([("y", "1")], inplace=False)
print(p_2["x_3|y=1"])

p_2["x_1|y=0"] = p_2["x_1,x_2,x_3|y"].marginalize(["x_2", "x_3"], inplace=False).reduce([("y", "0")], inplace=False)
print(p_2["x_1|y=0"])
p_2["x_2|y=0"] = p_2["x_1,x_2,x_3|y"].marginalize(["x_1", "x_3"], inplace=False).reduce([("y", "0")], inplace=False)
print(p_2["x_2|y=0"])
p_2["x_3|y=0"] = p_2["x_1,x_2,x_3|y"].marginalize(["x_1", "x_2"], inplace=False).reduce([("y", "0")], inplace=False)
print(p_2["x_3|y=0"])

y = 1:
+--------+------------+
| x_1    |   phi(x_1) |
+========+============+
| x_1(0) |     0.3924 |
+--------+------------+
| x_1(1) |     0.6076 |
+--------+------------+
+--------+------------+
| x_2    |   phi(x_2) |
+========+============+
| x_2(0) |     0.4561 |
+--------+------------+
| x_2(1) |     0.5439 |
+--------+------------+
+--------+------------+
| x_3    |   phi(x_3) |
+========+============+
| x_3(0) |     0.3710 |
+--------+------------+
| x_3(1) |     0.6290 |
+--------+------------+
+--------+------------+
| x_1    |   phi(x_1) |
+========+============+
| x_1(0) |     0.9091 |
+--------+------------+
| x_1(1) |     0.0909 |
+--------+------------+
+--------+------------+
| x_2    |   phi(x_2) |
+========+============+
| x_2(0) |     0.6667 |
+--------+------------+
| x_2(1) |     0.3333 |
+--------+------------+
+--------+------------+
| x_3    |   phi(x_3) |
+========+============+
| x_3(0) |     0.9901 |
+--------+------------+
| x_3(1) |     0.0099 |
+--------

2. Construct a noisy-OR model with $K=6$ and $\boldsymbol{\lambda} = (10^{-4}, 0.99, 0.9, 0.8, 0.7, 0.6, 0.5)$. Again, assume uniform priors for $x_k$ and $P(L=1)=1$.
    - For $\mathbf{x}=(0,0,0,0,0,1)$, what is the probability $p(y|\mathbf{x})$?
    - Let $\mathbf{x}=(1,0,0,0,0,1)$ What is the probability $p(y|\mathbf{x})$?

In [9]:
#Define variables and domains
variables = ["L", "x_1", "x_2", "x_3", "x_4", "x_5", "x_6", "z_0", "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "y"]
domains = {"L": ["0", "1"],
           "x_1": ["0", "1"],
           "x_2": ["0", "1"],
           "x_3": ["0", "1"],
           "x_4": ["0", "1"],
           "x_5": ["0", "1"],
           "x_6": ["0", "1"],
           "z_0": ["0", "1"],
           "z_1": ["0", "1"],
           "z_2": ["0", "1"],
           "z_3": ["0", "1"],
           "z_4": ["0", "1"],
           "z_5": ["0", "1"],
           "z_6": ["0", "1"],
           "y": ["0", "1"]}

p_3 = dict()    # use it to store out potentials

p_3["x_1"] = DiscreteFactor(variables=["x_1"],
                        cardinality=[len(domains["x_1"])],
                        values=[0.5, 0.5],
                        state_names=domains)

p_3["x_2"] = DiscreteFactor(variables=["x_2"],
                        cardinality=[len(domains["x_2"])],
                        values=[0.5, 0.5],
                        state_names=domains)

p_3["x_3"] = DiscreteFactor(variables=["x_3"],
                        cardinality=[len(domains["x_3"])],
                        values=[0.5, 0.5],
                        state_names=domains)

p_3["x_4"] = DiscreteFactor(variables=["x_4"],
                        cardinality=[len(domains["x_4"])],
                        values=[0.5, 0.5],
                        state_names=domains)

p_3["x_5"] = DiscreteFactor(variables=["x_5"],
                        cardinality=[len(domains["x_5"])],
                        values=[0.5, 0.5],
                        state_names=domains)

p_3["x_6"] = DiscreteFactor(variables=["x_6"],
                        cardinality=[len(domains["x_6"])],
                        values=[0.5, 0.5],
                        state_names=domains)

lambda_0 = 0.0001
lambda_1 = 0.99
lambda_2 = 0.9
lambda_3 = 0.8
lambda_4 = 0.7
lambda_5 = 0.6
lambda_6 = 0.5

p_3["L"] = DiscreteFactor(variables=["L"],
                        cardinality=[len(domains["L"])],
                        values=[0, 1],
                        state_names=domains)

p_3["z_0|L"] = DiscreteFactor(variables=["L", "z_0"],
                        cardinality=[len(domains["L"]), len(domains["z_0"])],
                        values=[1, 0, 1-lambda_0, lambda_0],
                        state_names=domains)  

p_3["z_1|x_1"] = DiscreteFactor(variables=["x_1", "z_1"],
                        cardinality=[len(domains["x_1"]), len(domains["z_1"])],
                        values=[1, 0, 1-lambda_1, lambda_1],
                        state_names=domains)

p_3["z_2|x_2"] = DiscreteFactor(variables=["x_2", "z_2"],
                        cardinality=[len(domains["x_2"]), len(domains["z_2"])],
                        values=[1, 0, 1-lambda_2, lambda_2],
                        state_names=domains)

p_3["z_3|x_3"] = DiscreteFactor(variables=["x_3", "z_3"],
                        cardinality=[len(domains["x_3"]), len(domains["z_3"])],
                        values=[1, 0, 1-lambda_3, lambda_3],
                        state_names=domains)

p_3["z_4|x_4"] = DiscreteFactor(variables=["x_4", "z_4"],
                        cardinality=[len(domains["x_4"]), len(domains["z_4"])],
                        values=[1, 0, 1-lambda_3, lambda_3],
                        state_names=domains)

p_3["z_5|x_5"] = DiscreteFactor(variables=["x_5", "z_5"],
                        cardinality=[len(domains["x_5"]), len(domains["z_5"])],
                        values=[1, 0, 1-lambda_3, lambda_3],
                        state_names=domains)

p_3["z_6|x_6"] = DiscreteFactor(variables=["x_6", "z_6"],
                        cardinality=[len(domains["x_6"]), len(domains["z_6"])],
                        values=[1, 0, 1-lambda_3, lambda_3],
                        state_names=domains)

p_3["y|z_0,z_1,z_2,z_3,z_4,z_5,z_6"] = DiscreteFactor(variables=["y", "z_0", "z_1", "z_2", "z_3", "z_4", "z_5", "z_6"],
                        cardinality=[len(domains["y"]), len(domains["z_0"]), len(domains["z_1"]), len(domains["z_2"]), len(domains["z_3"]), len(domains["z_4"]), len(domains["z_5"]), len(domains["z_6"])],
                        values=[1]+[0]*((256//2)-1)+[0]+[1]*((256//2)-1),
                        state_names=domains)

p_3["y,z_0,z_1,z_2,z_3,z_4,z_5,z_6,L,x_1,x_2,x_3,x_4,x_5,x_6"] = p_3["y|z_0,z_1,z_2,z_3,z_4,z_5,z_6"] * p_3["z_0|L"] * p_3["z_1|x_1"] * p_3["z_2|x_2"] * p_3["z_3|x_3"] * p_3["z_4|x_4"] * p_3["z_5|x_5"] * p_3["z_6|x_6"] * p_3["L"] * p_3["x_1"] * p_3["x_2"] * p_3["x_3"] * p_3["x_4"] * p_3["x_5"] * p_3["x_6"]
p_3["y,x_1,x_2,x_3,x_4,x_5,x_6"] = p_3["y,z_0,z_1,z_2,z_3,z_4,z_5,z_6,L,x_1,x_2,x_3,x_4,x_5,x_6"].marginalize(["L", "z_0", "z_1", "z_2","z_3", "z_4", "z_5","z_6"], inplace=False)

print("x = (0,0,0,0,0,1):")
p_3["y|x_1,x_2,x_3,x_4,x_5,x_6"] = p_3["y,x_1,x_2,x_3,x_4,x_5,x_6"]/p_3["x_1"]/p_3["x_2"]/p_3["x_3"]/p_3["x_4"]/p_3["x_5"]/p_3["x_6"]
P_1 = p_3["y|x_1,x_2,x_3,x_4,x_5,x_6"].reduce([("x_1", "0"), ("x_2", "0"), ("x_3", "0"), ("x_4", "0"), ("x_5", "0"), ("x_6", "1")], inplace=False)
print(P_1)
print("------------------------------------------------\nx = (1,0,0,0,0,1):")
P_2 = p_3["y|x_1,x_2,x_3,x_4,x_5,x_6"].reduce([("x_1", "1"), ("x_2", "0"), ("x_3", "0"), ("x_4", "0"), ("x_5", "0"), ("x_6", "1")], inplace=False)
print(P_2)

x = (0,0,0,0,0,1):
+------+----------+
| y    |   phi(y) |
+======+==========+
| y(0) |   0.2000 |
+------+----------+
| y(1) |   0.8000 |
+------+----------+
------------------------------------------------
x = (1,0,0,0,0,1):
+------+----------+
| y    |   phi(y) |
+======+==========+
| y(0) |   0.0020 |
+------+----------+
| y(1) |   0.9980 |
+------+----------+


## References

\[1\]  J. Pearl. Probabilistic reasoning in intelligent systems: networks of plausible inference. Elsevier, 2014.